In [3]:
import pandas as pd
import math
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import time
import os

In [ ]:
problem_files = ['savedrecs (76).txt', 'savedrecs - 2023-11-14T154913.176.txt', 
'savedrecs - 2023-11-17T045126.125.txt',
 'savedrecs - 2023-11-17T045221.722.txt',
 'savedrecs - 2023-11-17T045902.713.txt',
 'savedrecs - 2023-11-17T045957.587.txt',
 'savedrecs - 2023-11-17T052949.350.txt',
 'savedrecs - 2023-11-17T055806.508.txt']

In [24]:
range_journals = "400-499"
filnames = os.listdir(f"C:/Users/isabe/Desktop/Tese/journals{range_journals}")
journals_data = pd.DataFrame()
for file in filnames:
    try: 
        data = pd.read_csv(f"C:/Users/isabe/Desktop/Tese/journals{range_journals}/{file}", sep = '\t')
        journals_data = pd.concat([journals_data, data], ignore_index= True)
    except:
        problem_files.append(file)

In [25]:
problem_files

['savedrecs - 2023-11-17T045126.125.txt',
 'savedrecs - 2023-11-17T045221.722.txt',
 'savedrecs - 2023-11-17T045902.713.txt',
 'savedrecs - 2023-11-17T045957.587.txt',
 'savedrecs - 2023-11-17T052949.350.txt',
 'savedrecs - 2023-11-17T055806.508.txt',
 'savedrecs - 2023-11-19T142357.108.txt',
 'savedrecs - 2023-11-19T144918.959.txt',
 'savedrecs - 2023-11-19T191405.756.txt',
 'savedrecs - 2023-11-19T194002.578.txt',
 'savedrecs - 2023-11-19T213606.053.txt',
 'savedrecs - 2023-11-19T220356.736.txt',
 'savedrecs - 2023-11-19T232403.624.txt',
 'savedrecs - 2023-11-19T235136.435.txt']

In [27]:
folder_list =  [f"{i}-{i+99}" for i in range(100, 1701, 100)]
print(folder_list)

['100-199', '200-299', '300-399', '400-499', '500-599', '600-699', '700-799', '800-899', '900-999', '1000-1099', '1100-1199', '1200-1299', '1300-1399', '1400-1499', '1500-1599', '1600-1699', '1700-1799']


In [26]:
journals_data.to_csv(f'./retractions_data/journals_data{range_journals}.csv', index= False)

In [9]:
import random
list_of_objects = ['a', 'b', 'c', 'd']
random.shuffle(list_of_objects)
list_of_objects

['a', 'b', 'd', 'c']

`driver.find_element('xpath', value)`

xpath é um identificador do elemento. 2o param é o valor que eu quero: 
- ir ao website, right click, inspect, 
- botão de quadrado com seta (1o elemento da barra de cima), 
- clicar no elemento do website, aquilo faz highlight do código correspondente. 
- right click, copy xpath


`.click()` -> para clicar no website

`.send_keys("What I want to write")` -> para escrever dentro

In [16]:
def import_wos_data(link):
    concat_df = pd.DataFrame()

    #open browser
    service = Service(ChromeDriverManager().install()) #install the chrome driver correspondent to current chrome version
    driver = webdriver.Chrome(service= service)

    #go to website
    driver.get(link) #link should be the link of the query in wos
    time.sleep(10)

    # Reject cookies
    driver.find_element('xpath', '//*[@id="onetrust-reject-all-handler"]').click()
    #time.sleep(5)

    n_publications = int(driver.find_element(By.CLASS_NAME,"brand-blue").text.replace(',', ''))
    #print(n_publications)
    for page in range(math.ceil(n_publications/1000)):
        time.sleep(5)

        #last article number
        last_article = int(str(page+1)+'000')
        
        try:
            #click export to excel
            driver.find_element('xpath', '//*[@id="snRecListTop"]/app-export-menu/div/button').click()
            time.sleep(3)
            #driver.find_element('xpath', '//*[@id="exportToExcelButton"]').click()
            driver.find_element('xpath', '//*[@id="exportToTabWinButton"]').click() #export to tab delimited file
        except:
            print(f'Page number {last_article-1999} until {last_article-1000} was not exported')
            driver.find_element('xpath', '/html/body/app-wos/main/div/div/div[2]/div/div/div[2]/app-input-route[1]/app-export-overlay/div/div[3]/div[1]/button/span[1]/mat-icon/svg').click()
            continue

        time.sleep(5)

        #choose range of papers to export
        driver.find_element('xpath', '//*[@id="radio3"]/label/span[1]/span[2]').click()
        time.sleep(2)
        #Starting range
        driver.find_element(By.NAME, "markFrom").clear() 
        #time.sleep(1)
        driver.find_element(By.NAME, "markFrom").send_keys(last_article-999)
        #time.sleep(1)

        if last_article==(math.ceil(n_publications/1000)*1000):
            last_article==n_publications

        # End of range
        driver.find_element(By.NAME, "markTo").clear() 
        #time.sleep(1)
        driver.find_element(By.NAME, "markTo").send_keys(last_article)
        #time.sleep(1)

        #choose the record content to be "Full Record"
        driver.find_element('xpath', '/html/body/app-wos/main/div/div/div[2]/div/div/div[2]/app-input-route[1]/app-export-overlay/div/div[3]/div[2]/app-export-out-details/div/div[2]/form/div/div[1]/wos-select/button').click()
        time.sleep(2)
        driver.find_element('xpath', '//*[@id="global-select"]/div/div/div[3]').click()
        time.sleep(5)

        #click export
        driver.find_element('xpath', '/html/body/app-wos/main/div/div/div[2]/div/div/div[2]/app-input-route[1]/app-export-overlay/div/div[3]/div[2]/app-export-out-details/div/div[2]/form/div/div[2]/button[1]').click()

        time.sleep(12)

        #import the file and concatenate to create a single dataframe
        if page==0:
            #df = pd.read_excel(f'C:/Users/isabe/Downloads/savedrecs.xls')
            df = pd.read_csv(f'C:/Users/isabe/Downloads/savedrecs.txt',sep='\t')
            
        else:
            #df = pd.read_excel(f'C:/Users/isabe/Downloads/savedrecs ({page}).xls')
            df = pd.read_csv(f'C:/Users/isabe/Downloads/savedrecs ({page}).txt',sep='\t')
        
        concat_df = pd.concat([concat_df, df])
    
    driver.close()
    return concat_df

## Import data about papers analysing retractions

In [17]:
#retracted_articles = import_wos_data("https://www.webofscience.com/wos/woscc/summary/76c425c6-377e-4cd1-ad11-c5174fb61c16-a6056296/relevance/1")
#retracted_articles.to_excel('./papers_abt_retractions/retracted_articles.xlsx', index=False)
retracted_articles = pd.read_excel('./papers_abt_retractions/retracted_articles.xlsx')
retracted_articles.head()

Exception ignored in: <function Service.__del__ at 0x000001E3FBB149D0>
Traceback (most recent call last):
  File "c:\Users\isabe\Desktop\Tese\tese\lib\site-packages\selenium\webdriver\common\service.py", line 193, in __del__
    self.stop()
  File "c:\Users\isabe\Desktop\Tese\tese\lib\site-packages\selenium\webdriver\common\service.py", line 154, in stop
    self.send_remote_shutdown_command()
  File "c:\Users\isabe\Desktop\Tese\tese\lib\site-packages\selenium\webdriver\common\service.py", line 138, in send_remote_shutdown_command
    if not self.is_connectable():
  File "c:\Users\isabe\Desktop\Tese\tese\lib\site-packages\selenium\webdriver\common\service.py", line 127, in is_connectable
    return utils.is_connectable(self.port)
  File "c:\Users\isabe\Desktop\Tese\tese\lib\site-packages\selenium\webdriver\common\utils.py", line 101, in is_connectable
    socket_ = socket.create_connection((host, port), 1)
  File "c:\Users\isabe\Desktop\Tese\tese\lib\socket.py", line 833, in create_con

In [ ]:
retracted_articles = retracted_articles.drop('Unnamed: 0', axis=1)

In [ ]:
len(retracted_articles.columns)

71

In [ ]:
new_names={'PT': 'Publication Type',
'AU': 'Authors',
'BA': 'Book Authors',
'BE': 'Book Editors',
'GP': 'Book Group Authors',
'AF': 'Author Full Names',
'BF': 'Book Author Full Names',
'CA': 'Group Authors',
'TI': 'Article Title',
'SO': 'Source Title',
'SE': 'Book Series Title',
'BS': 'Book Series Subtitle',
'LA': 'Language',
'DT': 'Document Type',
'CT': 'Conference Title',
'CY': 'Conference Date',
'CL': 'Conference Location',
'SP': 'Conference Sponsor',
'HO': 'Conference Host',
'DE': 'Author Keywords',
'ID': 'Keywords Plus',
'AB': 'Abstract',
'C1': 'Addresses',
'C3': 'Affiliations', #
'RP': 'Reprint Addresses',
'EM': 'Email Addresses',
'RI': 'Researcher Ids',
'OI': 'ORCIDs',
'FU': 'Funding Orgs',
'FP': 'Funding Name Preferred', #
'FX': 'Funding Text',
'CR': 'Cited References',
'NR': 'Cited Reference Count',
'TC': 'Times Cited, WoS Core',
'Z9': 'Times Cited, All Databases',
'U1': '180 Day Usage Count',
'U2': 'Since 2013 Usage Count',
'PU': 'Publisher',
'PI': 'Publisher City',
'PA': 'Publisher Address',
'SN': 'ISSN',
'EI': 'eISSN',
'BN': 'ISBN',
'J9': 'Journal Abbreviation',
'JI': 'Journal ISO Abbreviation',
'PD': 'Publication Date',
'PY': 'Publication Year',
'VL': 'Volume',
'IS': 'Issue',
'PN': 'Part Number',
'SU': 'Supplement',
'SI': 'Special Issue',
'MA': 'Meeting Abstract',
'BP': 'Start Page',
'EP': 'End Page',
'AR': 'Article Number',
'DI': 'DOI',
'DL': 'DOI Link', #
'D2': 'Book DOI',
'EA': 'Early Access Date',
'PG': 'Number of Pages',
'WC': 'WoS Categories',
'WE': 'Web of Science Index', #
'SC': 'Research Areas',
'GA': 'IDS Number',
'PM': 'Pubmed Id',
'OA': 'Open Access Designations',
'HC': 'Highly Cited Status',
'HP': 'Hot Paper Status',
'DA': 'Date of Export',
'UT': 'UT (Unique WOS ID)'}


In [ ]:
retracted_articles.rename(columns=new_names, inplace=True)
retracted_articles.columns

Index(['Publication Type', 'Authors', 'Book Authors', 'Book Editors',
       'Book Group Authors', 'Author Full Names', 'Book Author Full Names',
       'Group Authors', 'Article Title', 'Source Title', 'Book Series Title',
       'Book Series Subtitle', 'Language', 'Document Type', 'Conference Title',
       'Conference Date', 'Conference Location', 'Conference Sponsor',
       'Conference Host', 'Author Keywords', 'Keywords Plus', 'Abstract',
       'Addresses', 'Affiliations', 'Reprint Addresses', 'Email Addresses',
       'Researcher Ids', 'ORCIDs', 'Funding Orgs', 'Funding Name Preferred',
       'Funding Text', 'Cited References', 'Cited Reference Count',
       'Times Cited, WoS Core', 'Times Cited, All Databases',
       '180 Day Usage Count', 'Since 2013 Usage Count', 'Publisher',
       'Publisher City', 'Publisher Address', 'ISSN', 'eISSN', 'ISBN',
       'Journal Abbreviation', 'Journal ISO Abbreviation', 'Publication Date',
       'Publication Year', 'Volume', 'Issue', 'Pa

In [ ]:
retracted_articles.head()

,Publication Type,Authors,Book Authors,Book Editors,Book Group Authors,Author Full Names,Book Author Full Names,Group Authors,Article Title,Source Title,...,WoS Categories,Web of Science Index,Research Areas,IDS Number,Pubmed Id,Open Access Designations,Highly Cited Status,Hot Paper Status,Date of Export,UT (Unique WOS ID)
0,J,"Halevi, G",NaN,NaN,NaN,"Halevi, Gali",NaN,NaN,Why Articles in Arts and Humanities Are Being ...,PUBLISHING RESEARCH QUARTERLY,...,Information Science & Library Science,Emerging Sources Citation Index (ESCI),Information Science & Library Science,KI5WT,NaN,NaN,NaN,NaN,2023-09-24,WOS:000500631500001
1,J,"Gungor, T; Basari, N",NaN,NaN,NaN,"Gungor, Tugba; Basari, Nimet",NaN,NaN,Bibliometric Examination of Retracted Articles...,TURKISH LIBRARIANSHIP,...,Information Science & Library Science,Emerging Sources Citation Index (ESCI),Information Science & Library Science,RH6WX,NaN,gold,NaN,NaN,2023-09-24,WOS:000636357400003
2,J,"Gholampour, B; Gholampour, S; Noruzi, A; Arsen...",NaN,NaN,NaN,"Gholampour, Behzad; Gholampour, Sajad; Noruzi,...",NaN,NaN,Retracted articles in oncology in the last thr...,SCIENTOMETRICS,...,"Computer Science, Interdisciplinary Applicatio...",Science Citation Index Expanded (SCI-EXPANDED)...,Computer Science; Information Science & Librar...,0M5LI,NaN,NaN,NaN,NaN,2023-09-24,WOS:000761851200002
3,J,"Leta, J; Araujo, K; Treiber, S",NaN,NaN,NaN,"Leta, Jacqueline; Araujo, Kizi; Treiber, Steph...",NaN,NaN,Citing documents of Wakefield's retracted arti...,SCIENTOMETRICS,...,"Computer Science, Interdisciplinary Applicatio...",Science Citation Index Expanded (SCI-EXPANDED)...,Computer Science; Information Science & Librar...,6U3BO,NaN,NaN,NaN,NaN,2023-09-24,WOS:000777873900002
4,J,"Dal-Re, R",NaN,NaN,NaN,"Dal-Re, Rafael",NaN,NaN,Analysis of biomedical Spanish articles retrac...,MEDICINA CLINICA,...,"Medicine, General & Internal",Science Citation Index Expanded (SCI-EXPANDED),General & Internal Medicine,LK4OR,31239080,NaN,NaN,NaN,2023-09-24,WOS:000530843500003


In [ ]:
#retracted_articles.to_excel('./papers_abt_retractions/retracted_articles.xlsx')

Intermediate step: delete all "savedrecs" from /Downloads, as the information is saved in a new excel file. 

In [ ]:
#retracted_publications= import_wos_data("https://www.webofscience.com/wos/woscc/summary/75966a73-21ac-4541-851e-a8e748e62b61-a605660b/relevance/1")
#retracted_publications.to_excel('./papers_abt_retractions/retracted_publications.xlsx', index=False)
retracted_publications = pd.read_excel('./papers_abt_retractions/retracted_publications.xlsx')
retracted_publications.head()

,Unnamed: 0,Publication Type,Authors,Book Authors,Book Editors,Book Group Authors,Author Full Names,Book Author Full Names,Group Authors,Article Title,...,Web of Science Index,Research Areas,IDS Number,Pubmed Id,Open Access Designations,Highly Cited Status,Hot Paper Status,Date of Export,UT (Unique WOS ID),Web of Science Record
0,0,J,"Rosenkrantz, AB",NaN,NaN,NaN,"Rosenkrantz, Andrew B.",NaN,NaN,Retracted Publications Within Radiology Journals,...,Science Citation Index Expanded (SCI-EXPANDED),"Radiology, Nuclear Medicine & Medical Imaging",DC9PH,26797347.0,NaN,NaN,NaN,2023-09-24,WOS:000369553000013,0
1,1,J,"Wang, T; Xing, QR; Wang, H; Chen, W",NaN,NaN,NaN,"Wang, Tao; Xing, Qin-Rui; Wang, Hui; Chen, Wei",NaN,NaN,Retracted Publications in the Biomedical Liter...,...,Science Citation Index Expanded (SCI-EXPANDED)...,Social Sciences - Other Topics; Engineering; H...,IF2VR,29516389.0,NaN,NaN,NaN,2023-09-24,WOS:000472938700011,0
2,2,J,"Mansourzadeh, MJ; Ghazimirsaeid, J; Motamedi, ...",NaN,NaN,NaN,"Mansourzadeh, Mohammad Javad; Ghazimirsaeid, J...",NaN,NaN,A Survey of Iranian Retracted Publications Ind...,...,Science Citation Index Expanded (SCI-EXPANDED)...,"Public, Environmental & Occupational Health",PQ8XQ,34178778.0,"Green Published, gold",NaN,NaN,2023-09-24,WOS:000606826700021,0
3,3,J,"Feng, LZ; Yuan, JP; Yang, LY",NaN,NaN,NaN,"Feng, Lingzi; Yuan, Junpeng; Yang, Liying",NaN,NaN,An observation framework for retracted publica...,...,Science Citation Index Expanded (SCI-EXPANDED)...,Computer Science; Information Science & Librar...,OT3PI,NaN,NaN,NaN,NaN,2023-09-24,WOS:000577239100003,0
4,4,J,"Chen, W; Xing, QR; Wang, H; Wang, T",NaN,NaN,NaN,"Chen, Wei; Xing, Qin-Rui; Wang, Hui; Wang, Tao",NaN,NaN,Retracted publications in the biomedical liter...,...,Science Citation Index Expanded (SCI-EXPANDED)...,Computer Science; Information Science & Librar...,FR9MN,NaN,NaN,NaN,NaN,2023-09-24,WOS:000419397900009,0


In [ ]:
len(retracted_publications.columns)

73

In [ ]:
#retracted_publications.to_excel('./papers_abt_retractions/retracted_publications.xlsx')

In [ ]:
#retracted_papers = import_wos_data("https://www.webofscience.com/wos/woscc/summary/0de7a5f8-6b08-4323-8b1f-de604c297616-a6056816/relevance/1")
#retracted_papers.to_excel('./papers_abt_retractions/retracted_papers.xlsx', index=False)
retracted_papers = pd.read_excel('./papers_abt_retractions/retracted_papers.xlsx')
retracted_papers.head()

,Publication Type,Authors,Book Authors,Book Editors,Book Group Authors,Author Full Names,Book Author Full Names,Group Authors,Article Title,Source Title,...,Web of Science Index,Research Areas,IDS Number,Pubmed Id,Open Access Designations,Highly Cited Status,Hot Paper Status,Date of Export,UT (Unique WOS ID),Web of Science Record
0,J,"Panahi, S; Soleimanpour, S",NaN,NaN,NaN,"Panahi, Sirous; Soleimanpour, Samira",NaN,NaN,"The landscape of the characteristics, citation...",ACCOUNTABILITY IN RESEARCH-ETHICS INTEGRITY AN...,...,Science Citation Index Expanded (SCI-EXPANDED),Medical Ethics,N5PZ8,34612782.0,NaN,NaN,NaN,2023-09-24,WOS:000709185600001,0
1,J,"da Silva, JAT; Dobranszki, J",NaN,NaN,NaN,"da Silva, Jaime A. Teixeira; Dobranszki, Judit",NaN,NaN,Highly cited retracted papers,SCIENTOMETRICS,...,Science Citation Index Expanded (SCI-EXPANDED)...,Computer Science; Information Science & Librar...,EL0VI,NaN,NaN,NaN,NaN,2023-09-24,WOS:000394339500030,0
2,J,"Candal-Pedreira, C; Ross, JS; Ruano-Ravina, A;...",NaN,NaN,NaN,"Candal-Pedreira, Cristina; Ross, Joseph S.; Ru...",NaN,NaN,Retracted papers originating from paper mills:...,BMJ-BRITISH MEDICAL JOURNAL,...,Science Citation Index Expanded (SCI-EXPANDED),General & Internal Medicine,6T3GK,36442874.0,"Green Published, hybrid",NaN,NaN,2023-09-24,WOS:000893566800005,0
3,J,"Peng, H; Romero, DM; Horvat, EA",NaN,NaN,NaN,"Peng, Hao; Romero, Daniel M.; Horvat, Emoke-Agnes",NaN,NaN,Dynamics of cross-platform attention to retrac...,PROCEEDINGS OF THE NATIONAL ACADEMY OF SCIENCE...,...,Science Citation Index Expanded (SCI-EXPANDED),Science & Technology - Other Topics,3R1VC,35700358.0,"Green Submitted, hybrid, Green Published",NaN,NaN,2023-09-24,WOS:000838706700008,0
4,J,"Shepperd, M; Yousefi, L",NaN,NaN,NaN,"Shepperd, Martin; Yousefi, Leila",NaN,NaN,An analysis of retracted papers in Computer Sc...,PLOS ONE,...,Science Citation Index Expanded (SCI-EXPANDED),Science & Technology - Other Topics,F8JN8,37159472.0,"Green Submitted, gold, Green Published",NaN,NaN,2023-09-24,WOS:000984757900018,0


In [ ]:
len(retracted_papers.columns)

72

In [ ]:
#retracted_papers.to_excel('./papers_abt_retractions/retracted_papers.xlsx')

### Joining data

In [ ]:
abt_retractions = pd.concat([retracted_papers, retracted_publications, retracted_articles])
abt_retractions.info()

<class 'pandas.core.frame.DataFrame'>
Index: 17241 entries, 0 to 15195
Data columns (total 73 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Publication Type            17241 non-null  object 
 1   Authors                     17237 non-null  object 
 2   Book Authors                42 non-null     object 
 3   Book Editors                1436 non-null   object 
 4   Book Group Authors          851 non-null    object 
 5   Author Full Names           17237 non-null  object 
 6   Book Author Full Names      42 non-null     object 
 7   Group Authors               48 non-null     object 
 8   Article Title               17241 non-null  object 
 9   Source Title                17241 non-null  object 
 10  Book Series Title           1485 non-null   object 
 11  Book Series Subtitle        0 non-null      float64
 12  Language                    17241 non-null  object 
 13  Document Type               17241 no

In [ ]:
abt_retractions.drop_duplicates(subset='UT (Unique WOS ID)',inplace=True)

In [ ]:
abt_retractions.info()

<class 'pandas.core.frame.DataFrame'>
Index: 16530 entries, 0 to 15194
Data columns (total 73 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Publication Type            16530 non-null  object 
 1   Authors                     16526 non-null  object 
 2   Book Authors                42 non-null     object 
 3   Book Editors                1420 non-null   object 
 4   Book Group Authors          845 non-null    object 
 5   Author Full Names           16526 non-null  object 
 6   Book Author Full Names      42 non-null     object 
 7   Group Authors               42 non-null     object 
 8   Article Title               16530 non-null  object 
 9   Source Title                16530 non-null  object 
 10  Book Series Title           1469 non-null   object 
 11  Book Series Subtitle        0 non-null      float64
 12  Language                    16530 non-null  object 
 13  Document Type               16530 no

In [ ]:
abt_retractions['Document Type'].value_counts()

Document Type
Article; Retracted Publication                             9544
Proceedings Paper; Retracted Publication                   1982
Article                                                    1981
Review                                                      561
Retraction                                                  535
Review; Retracted Publication                               364
Proceedings Paper                                           305
Retraction; Retracted Publication                           158
Article; Proceedings Paper; Retracted Publication           148
Letter; Retracted Publication                               137
Article; Proceedings Paper                                  129
Editorial Material                                          108
Editorial Material; Retracted Publication                    85
Correction; Retracted Publication                            82
News Item                                                    75
Correction                

In [ ]:
abt_retractions.columns

Index(['Publication Type', 'Authors', 'Book Authors', 'Book Editors',
       'Book Group Authors', 'Author Full Names', 'Book Author Full Names',
       'Group Authors', 'Article Title', 'Source Title', 'Book Series Title',
       'Book Series Subtitle', 'Language', 'Document Type', 'Conference Title',
       'Conference Date', 'Conference Location', 'Conference Sponsor',
       'Conference Host', 'Author Keywords', 'Keywords Plus', 'Abstract',
       'Addresses', 'Affiliations', 'Reprint Addresses', 'Email Addresses',
       'Researcher Ids', 'ORCIDs', 'Funding Orgs', 'Funding Name Preferred',
       'Funding Text', 'Cited References', 'Cited Reference Count',
       'Times Cited, WoS Core', 'Times Cited, All Databases',
       '180 Day Usage Count', 'Since 2013 Usage Count', 'Publisher',
       'Publisher City', 'Publisher Address', 'ISSN', 'eISSN', 'ISBN',
       'Journal Abbreviation', 'Journal ISO Abbreviation', 'Publication Date',
       'Publication Year', 'Volume', 'Issue', 'Pa

In [ ]:
abt_retractions = abt_retractions[abt_retractions['Keywords Plus'] != "RETRACTED ARTICLE. SEE"]

In [ ]:
abt_retractions1 = abt_retractions[~abt_retractions['Document Type'].str.contains(r'retraction|retracted|withdrawal', case=False)]
abt_retractions1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3346 entries, 0 to 15194
Data columns (total 73 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Publication Type            3346 non-null   object 
 1   Authors                     3346 non-null   object 
 2   Book Authors                0 non-null      object 
 3   Book Editors                185 non-null    object 
 4   Book Group Authors          151 non-null    object 
 5   Author Full Names           3346 non-null   object 
 6   Book Author Full Names      0 non-null      object 
 7   Group Authors               11 non-null     object 
 8   Article Title               3346 non-null   object 
 9   Source Title                3346 non-null   object 
 10  Book Series Title           217 non-null    object 
 11  Book Series Subtitle        0 non-null      float64
 12  Language                    3346 non-null   object 
 13  Document Type               3346 non-

In [ ]:
possible_doc_types = ['Article', 'Review']
abt_retractions1 = abt_retractions[abt_retractions['Document Type']=='Article']
abt_retractions1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1981 entries, 0 to 15194
Data columns (total 73 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Publication Type            1981 non-null   object 
 1   Authors                     1981 non-null   object 
 2   Book Authors                0 non-null      object 
 3   Book Editors                1 non-null      object 
 4   Book Group Authors          0 non-null      object 
 5   Author Full Names           1981 non-null   object 
 6   Book Author Full Names      0 non-null      object 
 7   Group Authors               5 non-null      object 
 8   Article Title               1981 non-null   object 
 9   Source Title                1981 non-null   object 
 10  Book Series Title           1 non-null      object 
 11  Book Series Subtitle        0 non-null      float64
 12  Language                    1981 non-null   object 
 13  Document Type               1981 non-

In [ ]:
abt_retractions = abt_retractions[abt_retractions['Publication Year']<2023]
abt_retractions.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1919 entries, 1 to 15194
Data columns (total 73 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Publication Type            1919 non-null   object 
 1   Authors                     1919 non-null   object 
 2   Book Authors                0 non-null      object 
 3   Book Editors                1 non-null      object 
 4   Book Group Authors          0 non-null      object 
 5   Author Full Names           1919 non-null   object 
 6   Book Author Full Names      0 non-null      object 
 7   Group Authors               5 non-null      object 
 8   Article Title               1919 non-null   object 
 9   Source Title                1919 non-null   object 
 10  Book Series Title           1 non-null      object 
 11  Book Series Subtitle        0 non-null      float64
 12  Language                    1919 non-null   object 
 13  Document Type               1919 non-

In [ ]:
abt_retractions.describe().T

,count,mean,std,min,25%,50%,75%,max
Book Series Subtitle,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Cited References,1.0,42.000000,NaN,42.0,42.00,42.0,42.0,42.0
Cited Reference Count,1919.0,40.368421,34.754558,0.0,21.00,33.0,50.0,724.0
"Times Cited, WoS Core",1919.0,32.675873,76.161874,0.0,3.00,10.0,30.0,1102.0
"Times Cited, All Databases",1919.0,34.244398,79.135140,0.0,3.00,11.0,31.0,1138.0
180 Day Usage Count,1919.0,1.049505,2.906278,0.0,0.00,0.0,1.0,54.0
Publication Year,1919.0,2010.891089,44.681635,82.0,2006.00,2013.0,2019.0,2022.0
Web of Science Record,945.0,0.000000,0.000000,0.0,0.00,0.0,0.0,0.0
Unnamed: 0,118.0,215.135593,144.108691,0.0,79.25,209.0,331.0,601.0


In [ ]:
abt_retractions = abt_retractions[abt_retractions['Publication Year']!=abt_retractions['Publication Year'].min()]

In [ ]:
abt_retractions.to_excel('./papers_abt_retractions/abt_retractions.xlsx')

## Import data about retracted papers

In [ ]:
retractions = import_wos_data('https://www.webofscience.com/wos/woscc/summary/53c1697c-1d2a-4502-a44e-f1230bd60527-a6305b44/relevance/1')
#retractions = pd.read_excel('./retractions_data/wos_retractions.xlsx')
retractions.head()

,PT,AU,BA,BE,GP,AF,BF,CA,TI,SO,...,WC,WE,SC,GA,PM,OA,HC,HP,DA,UT
0,J,"Albarghouth, MH; Hamza, A",NaN,NaN,NaN,"Albarghouth, Mohamad Hatem; Hamza, Amir",NaN,NaN,RETRACTION: Statins significantly repress rota...,GUT MICROBES,...,Gastroenterology & Hepatology; Microbiology,Science Citation Index Expanded (SCI-EXPANDED),Gastroenterology & Hepatology; Microbiology,6P2AT,NaN,"Green Published, gold",NaN,NaN,2023-09-24,WOS:000890736300001
1,J,"Anari, E; Akbarzadeh, A; Zarghami, N",NaN,NaN,NaN,"Anari, Elham; Akbarzadeh, Abolfazl; Zarghami, ...",NaN,NaN,RETRACTION: Chrysin-loaded PLGA-PEG nanopartic...,ARTIFICIAL CELLS NANOMEDICINE AND BIOTECHNOLOGY,...,Biotechnology & Applied Microbiology; Engineer...,Science Citation Index Expanded (SCI-EXPANDED),Biotechnology & Applied Microbiology; Engineer...,3F7XP,NaN,gold,NaN,NaN,2023-09-24,WOS:000830877600001
2,J,"Asadi, N; Annabi, N; Mostafavi, E; Anzabi, M; ...",NaN,NaN,NaN,"Asadi, Nahideh; Annabi, Nasim; Mostafavi, Ebra...",NaN,NaN,"RETRACTION: Synthesis, characterization and in...",ARTIFICIAL CELLS NANOMEDICINE AND BIOTECHNOLOGY,...,Biotechnology & Applied Microbiology; Engineer...,Science Citation Index Expanded (SCI-EXPANDED),Biotechnology & Applied Microbiology; Engineer...,0T6WV,NaN,gold,NaN,NaN,2023-09-24,WOS:000787107700001
3,J,"Das Adhikari, M; Jung, WY; Kim, JM; Yum, SG",NaN,NaN,NaN,"Das Adhikari, Manik; Jung, Woo-Young; Kim, Ji-...",NaN,NaN,RETRACTED: Seismic site characterization in no...,GEOMATICS NATURAL HAZARDS & RISK,...,"Geosciences, Multidisciplinary; Meteorology & ...",Science Citation Index Expanded (SCI-EXPANDED),Geology; Meteorology & Atmospheric Sciences; W...,ZC3NS,NaN,gold,NaN,NaN,2023-09-24,WOS:000757431500001
4,J,"Das Adhikari, M; Jung, WY; Kim, JM; Yum, SG",NaN,NaN,NaN,"Das Adhikari, Manik; Jung, Woo-Young; Kim, Ji-...",NaN,NaN,RETRACTION: Seismic site characterization in n...,GEOMATICS NATURAL HAZARDS & RISK,...,"Geosciences, Multidisciplinary; Meteorology & ...",Science Citation Index Expanded (SCI-EXPANDED),Geology; Meteorology & Atmospheric Sciences; W...,2I4LO,NaN,gold,NaN,NaN,2023-09-24,WOS:000814951800001


In [ ]:
retractions.info()

<class 'pandas.core.frame.DataFrame'>
Index: 24143 entries, 0 to 185
Data columns (total 71 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Publication Type            24143 non-null  object 
 1   Authors                     23917 non-null  object 
 2   Book Authors                43 non-null     object 
 3   Book Editors                1253 non-null   object 
 4   Book Group Authors          697 non-null    object 
 5   Author Full Names           23917 non-null  object 
 6   Book Author Full Names      43 non-null     object 
 7   Group Authors               346 non-null    object 
 8   Article Title               24143 non-null  object 
 9   Source Title                24143 non-null  object 
 10  Book Series Title           1273 non-null   object 
 11  Book Series Subtitle        0 non-null      float64
 12  Language                    24143 non-null  object 
 13  Document Type               24143 non-

In [ ]:
retractions.rename(columns=new_names, inplace=True)
retractions.to_excel('./retractions_data/wos_retractions.xlsx', index=False)
retractions.columns

Index(['Publication Type', 'Authors', 'Book Authors', 'Book Editors',
       'Book Group Authors', 'Author Full Names', 'Book Author Full Names',
       'Group Authors', 'Article Title', 'Source Title', 'Book Series Title',
       'Book Series Subtitle', 'Language', 'Document Type', 'Conference Title',
       'Conference Date', 'Conference Location', 'Conference Sponsor',
       'Conference Host', 'Author Keywords', 'Keywords Plus', 'Abstract',
       'Addresses', 'Affiliations', 'Reprint Addresses', 'Email Addresses',
       'Researcher Ids', 'ORCIDs', 'Funding Orgs', 'Funding Name Preferred',
       'Funding Text', 'Cited References', 'Cited Reference Count',
       'Times Cited, WoS Core', 'Times Cited, All Databases',
       '180 Day Usage Count', 'Since 2013 Usage Count', 'Publisher',
       'Publisher City', 'Publisher Address', 'ISSN', 'eISSN', 'ISBN',
       'Journal Abbreviation', 'Journal ISO Abbreviation', 'Publication Date',
       'Publication Year', 'Volume', 'Issue', 'Pa

In [ ]:
retractions.describe().T

,count,mean,std,min,25%,50%,75%,max
Book Series Subtitle,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
180 Day Usage Count,24142.0,0.552233,1.629891,0.0,0.0,0.0,1.0,105.0
Publication Year,24143.0,2016.086650,22.013822,6.0,2013.0,2018.0,2021.0,2022.0


In [ ]:
retractions['Publication Date'].value_counts()

Publication Date
DEC            1230
JUN            1041
AUG            1030
SEP             980
NOV             975
               ... 
2021 AUG 2        1
2021 JUL 28       1
2021 JUL 24       1
2022 DEC 30       1
FEB 29            1
Name: count, Length: 576, dtype: int64

# All articles

In [2]:
from selenium.webdriver.support import expected_conditions as EC
import os
from selenium.common.exceptions import NoAlertPresentException

from selenium.webdriver.support.wait import WebDriverWait
import pandas as pd
import math
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import time

In [3]:
journals = pd.read_csv('../scimagojr_2022.csv', sep=';')
journals

,Rank,Sourceid,Title,Type,Issn,SJR,SJR Best Quartile,H index,Total Docs. (2022),Total Docs. (3years),...,Total Cites (3years),Citable Docs. (3years),Cites / Doc. (2years),Ref. / Doc.,Country,Region,Publisher,Coverage,Categories,Areas
0,1,28773,Ca-A Cancer Journal for Clinicians,journal,"15424863, 00079235","86,091",Q1,198,44,118,...,30318,85,"299,99","97,00",United States,Northern America,Wiley-Blackwell,1950-2022,Hematology (Q1); Oncology (Q1),Medicine
1,2,29431,Quarterly Journal of Economics,journal,"00335533, 15314650","36,730",Q1,292,36,122,...,2141,122,"14,83","66,61",United Kingdom,Western Europe,Oxford University Press,1886-2022,Economics and Econometrics (Q1),"Economics, Econometrics and Finance"
2,3,20315,Nature Reviews Molecular Cell Biology,journal,"14710072, 14710080","34,201",Q1,485,121,328,...,13331,156,"35,47","89,29",United Kingdom,Western Europe,Nature Publishing Group,2000-2022,Cell Biology (Q1); Molecular Biology (Q1),"Biochemistry, Genetics and Molecular Biology"
3,4,18434,Cell,journal,"00928674, 10974172","26,494",Q1,856,420,1637,...,67791,1440,"43,80","65,74",United States,Northern America,Cell Press,1974-2022,"Biochemistry, Genetics and Molecular Biology (...","Biochemistry, Genetics and Molecular Biology"
4,5,15847,New England Journal of Medicine,journal,"00284793, 15334406","26,015",Q1,1130,1410,4561,...,133956,1854,"33,93","10,21",United States,Northern America,Massachussetts Medical Society,1945-2022,Medicine (miscellaneous) (Q1),Medicine
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18031,18032,17500154901,Progress in Molecular Biology and Translationa...,journal,"18771173, 18780814",NaN,-,110,90,314,...,1170,0,"2,72","108,88",Netherlands,Western Europe,Academic Press Inc.,2008-2022,Molecular Biology; Molecular Medicine,"Biochemistry, Genetics and Molecular Biology"
18032,18033,25192,Reviews of Environmental Contamination and Tox...,journal,01795953,NaN,-,94,22,99,...,441,0,"4,40","151,68",United States,Northern America,Springer New York,1987-2022,"Health, Toxicology and Mutagenesis; Medicine (...",Environmental Science; Medicine
18033,18034,5700155185,Voprosy Istorii (discontinued),journal,00428779,NaN,-,5,168,1316,...,38,1316,"0,03","7,96",Russian Federation,Eastern Europe,"Rossiiskaya Akademiya Nauk, Institut Istorii (...","1965, 1972, 1975, 1978-1982, 1985, 1988, 1999-...",History; Medicine (miscellaneous),Arts and Humanities; Medicine
18034,18035,21100873483,Wisdom (discontinued),journal,18293824,NaN,-,7,66,180,...,76,180,"0,45","25,61",Armenia,Eastern Europe,Khachatur Abovyan Armenian State Pedagogical U...,2018-2022,Philosophy,Arts and Humanities


In [4]:
# Calculate the threshold for the top 10%
threshold = int(0.10 * len(journals))

# Select the top 10% of rows
top_10_percent = journals.head(threshold)
top_10_percent

,Rank,Sourceid,Title,Type,Issn,SJR,SJR Best Quartile,H index,Total Docs. (2022),Total Docs. (3years),...,Total Cites (3years),Citable Docs. (3years),Cites / Doc. (2years),Ref. / Doc.,Country,Region,Publisher,Coverage,Categories,Areas
0,1,28773,Ca-A Cancer Journal for Clinicians,journal,"15424863, 00079235","86,091",Q1,198,44,118,...,30318,85,"299,99","97,00",United States,Northern America,Wiley-Blackwell,1950-2022,Hematology (Q1); Oncology (Q1),Medicine
1,2,29431,Quarterly Journal of Economics,journal,"00335533, 15314650","36,730",Q1,292,36,122,...,2141,122,"14,83","66,61",United Kingdom,Western Europe,Oxford University Press,1886-2022,Economics and Econometrics (Q1),"Economics, Econometrics and Finance"
2,3,20315,Nature Reviews Molecular Cell Biology,journal,"14710072, 14710080","34,201",Q1,485,121,328,...,13331,156,"35,47","89,29",United Kingdom,Western Europe,Nature Publishing Group,2000-2022,Cell Biology (Q1); Molecular Biology (Q1),"Biochemistry, Genetics and Molecular Biology"
3,4,18434,Cell,journal,"00928674, 10974172","26,494",Q1,856,420,1637,...,67791,1440,"43,80","65,74",United States,Northern America,Cell Press,1974-2022,"Biochemistry, Genetics and Molecular Biology (...","Biochemistry, Genetics and Molecular Biology"
4,5,15847,New England Journal of Medicine,journal,"00284793, 15334406","26,015",Q1,1130,1410,4561,...,133956,1854,"33,93","10,21",United States,Northern America,Massachussetts Medical Society,1945-2022,Medicine (miscellaneous) (Q1),Medicine
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1798,1799,19900192175,Journal of Topology,journal,"17538416, 17538424","1,575",Q1,28,56,119,...,137,119,"1,11","36,68",United Kingdom,Western Europe,John Wiley and Sons Ltd,2010-2022,Geometry and Topology (Q1),Mathematics
1799,1800,21100875643,Materials Chemistry Frontiers,journal,20521537,"1,575",Q1,70,367,1245,...,8782,1233,"7,21","63,92",United Kingdom,Western Europe,Royal Society of Chemistry,2017-2022,Materials Chemistry (Q1); Materials Science (m...,Materials Science
1800,1801,15061,Agricultural Systems,journal,"0308521X, 18732267","1,574",Q1,126,166,607,...,4408,601,"7,02","72,49",United Kingdom,Western Europe,Elsevier BV,1976-2022,Agronomy and Crop Science (Q1); Animal Science...,Agricultural and Biological Sciences
1801,1802,26112,Frontiers of Hormone Research,journal,03013073,"1,574",Q1,42,0,45,...,156,41,"0,92","0,00",Switzerland,Western Europe,S. Karger AG,"1975, 1977, 1984, 1996-1997, 1999-2002, 2004-2...","Endocrinology (Q1); Endocrinology, Diabetes an...","Biochemistry, Genetics and Molecular Biology; ..."


In [5]:
top_10_percent.columns

Index(['Rank', 'Sourceid', 'Title', 'Type', 'Issn', 'SJR', 'SJR Best Quartile',
       'H index', 'Total Docs. (2022)', 'Total Docs. (3years)', 'Total Refs.',
       'Total Cites (3years)', 'Citable Docs. (3years)',
       'Cites / Doc. (2years)', 'Ref. / Doc.', 'Country', 'Region',
       'Publisher', 'Coverage', 'Categories', 'Areas'],
      dtype='object')

In [6]:
unique_values = list(top_10_percent['Title'].values)

In [7]:
# Calculate the number of values per list
values_per_list = 100
num_lists = 18
values_in_last_list = len(unique_values) - (values_per_list * (num_lists - 1))

# Create the lists
list_of_journals = [unique_values[i:i + values_per_list] for i in range(0, len(unique_values) - values_in_last_list, values_per_list)]
list_of_journals.append(unique_values[-values_in_last_list:])

# Print the lists
for i, lst in enumerate(list_of_journals):
    print(f"List {i + 1}: {len(lst)} values - {lst}")

List 1: 100 values - ['Ca-A Cancer Journal for Clinicians', 'Quarterly Journal of Economics', 'Nature Reviews Molecular Cell Biology', 'Cell', 'New England Journal of Medicine', 'Nature Medicine', 'MMWR Recommendations and Reports', 'Nature Biotechnology', 'Nature Reviews Materials', 'American Economic Review', 'Nature Reviews Cancer', 'Nature', 'Journal of Political Economy', 'Reviews of Modern Physics', 'Nature Energy', 'Nature Reviews Genetics', 'Nature Reviews Drug Discovery', 'Chemical Reviews', 'Nature Reviews Immunology', 'Morbidity and Mortality Weekly Report', 'Nature Genetics', 'Administrative Science Quarterly', 'Academy of Management Annals', 'Journal of Finance', 'Annual Review of Immunology', 'MMWR Surveillance Summaries', 'Chemical Society Reviews', 'Nature Reviews Clinical Oncology', 'Immunity', 'Lancet, The', 'Review of Economic Studies', 'Nature Methods', 'World Psychiatry', 'IEEE Communications Surveys and Tutorials', 'Nature Materials', 'Foundations and Trends in Ma

In [8]:
link = 'https://www.scopus.com/results/results.uri?sort=plf-f&src=s&st1=%22retracted+articles%22&sid=ace58a9ad0b17eaf5a1f704503783b22&sot=b&sdt=b&sl=35&s=SRCTITLE%28%22American+Journal+of+Hematology%22%29&origin=searchbasic&editSaveSearch=&sessionSearchId=ace58a9ad0b17eaf5a1f704503783b22&limit=10&yearFrom=Before+1960&yearTo=Present&offset=10'

In [12]:
def import_scopus(link, journal):
    #clear journal name and change it
        driver.find_element('xpath','//*[@id="pendo-main-search-bar"]/div/div/label/input').clear()
        driver.find_element('xpath','//*[@id="pendo-main-search-bar"]/div/div/label/input').send_keys(f'"{journal}"')

        driver.implicitly_wait(15) 
        #open source title filter
        driver.find_element('xpath','//*[@id="container"]/micro-ui/document-search-results-page/div[1]/section[2]/div/div[1]/div[2]/div/div[2]/div/els-focus-tab-detector[8]/div/button').click()
        driver.find_element('xpath', '//*[@id="container"]/micro-ui/document-search-results-page/div[1]/section[1]/div[1]/div[2]/div/form/div[2]/div[2]/button[2]').click()

        #get number of publications in filter
        time.sleep(5)
        n_publications = driver.find_elements(By.TAG_NAME, 'h2')[0].text.replace(',', '')
        n_publications = int(n_publications.replace(' documents found', ''))

        for page in range(math.ceil(n_publications/20_000)):
            last_article = int(str(page+1)+'000')*20

            #click export button, choose csv
            driver.find_element('xpath', '//*[@id="container"]/micro-ui/document-search-results-page/div[1]/section[2]/div/div[2]/div/div[2]/div/div[1]/table/tbody/tr/td[2]/div/div/div[1]/span/button').click()
            driver.find_elements(By.CLASS_NAME, "Typography-module__lVnit")[52].click()

            # choose range

            driver.find_element('xpath', "//input[@type='radio' and @name='select-current-page' and not(@checked)]").click()
             # start
            driver.find_element('xpath', '//*[@id="container"]/micro-ui/document-search-results-page/div[1]/section[2]/div/div[2]/div/div[2]/div/div[1]/table/tbody/tr/td[2]/div/div/div[2]/div/div/section/div[1]/div/div/div[1]/div/div/div/div/div/div/div[1]/div/label/input').clear()
            driver.find_element('xpath', '//*[@id="container"]/micro-ui/document-search-results-page/div[1]/section[2]/div/div[2]/div/div[2]/div/div[1]/table/tbody/tr/td[2]/div/div/div[2]/div/div/section/div[1]/div/div/div[1]/div/div/div/div/div/div/div[1]/div/label/input').send_keys(last_article-19_999)
             # end
            if page==(math.ceil(n_publications/20_000)-1):
                last_article=n_publications
            driver.find_element('xpath', '//*[@id="container"]/micro-ui/document-search-results-page/div[1]/section[2]/div/div[2]/div/div[2]/div/div[1]/table/tbody/tr/td[2]/div/div/div[2]/div/div/section/div[1]/div/div/div[1]/div/div/div/div/div/div/div[2]/div/label/input').clear()
            driver.find_element('xpath', '//*[@id="container"]/micro-ui/document-search-results-page/div[1]/section[2]/div/div[2]/div/div[2]/div/div[1]/table/tbody/tr/td[2]/div/div/div[2]/div/div/section/div[1]/div/div/div[1]/div/div/div/div/div/div/div[2]/div/label/input').send_keys(last_article)
            
            # Select all information
            driver.find_element('xpath', '//*[@id="container"]/micro-ui/document-search-results-page/div[1]/section[2]/div/div[2]/div/div[2]/div/div[1]/table/tbody/tr/td[2]/div/div/div[2]/div/div/section/div[2]/div/div/span[1]/div/button/span').click()

            # export all information
            driver.find_element('xpath', '//*[@id="container"]/micro-ui/document-search-results-page/div[1]/section[2]/div/div[2]/div/div[2]/div/div[1]/table/tbody/tr/td[2]/div/div/div[2]/div/div/section/div[2]/div/div/span[2]/div/div/button/span[1]/div').click()
            
            

In [13]:
# Define a custom condition for WebDriverWait
def file_exists(_):
    return os.path.exists(file_path)

In [15]:
concat_df = pd.DataFrame()
loop_count= 0

#open browser
service = Service(ChromeDriverManager().install()) #install the chrome driver correspondent to current chrome version
driver = webdriver.Chrome(service= service)
wait = WebDriverWait(driver, 2000)

#go to website
#driver.get(link) #link should be the link of the query in wos
#driver.implicitly_wait(15)

#close pop-up
#driver.find_element('xpath','//*[@id="pendo-close-guide-2e51a252"]').click()
#driver.implicitly_wait(15) 

for journal in list_of_journals[0]:
    import_scopus(link, journal)

    if loop_count == 0:
        file_path = 'C:/Users/isabe/Downloads/scopus.csv'
        # Wait up to 60 seconds for the file to be downloaded
        WebDriverWait(driver, 300).until(file_exists)
        df = pd.read_csv(file_path)
                
    else:
        file_path = f'C:/Users/isabe/Downloads/scopus ({loop_count}).csv'
        WebDriverWait(driver, 300).until(file_exists)
        df = pd.read_csv(file_path)
                
    loop_count += 1
    concat_df = pd.concat([concat_df, df])

TypeError: 'WebElement' object is not subscriptable

<span style="background-color:red;">Stop Running here.</span>

In [19]:
def import_scopus(link, journals):
    loop_count= 0
    concat_df = pd.DataFrame()
    #open browser
    service = Service(ChromeDriverManager().install()) #install the chrome driver correspondent to current chrome version
    driver = webdriver.Chrome(service= service)
    wait = WebDriverWait(driver, 2000)

    #go to website
    driver.get(link) #link should be the link of the query in wos
    driver.implicitly_wait(15)

    #close pop-up
    driver.find_element('xpath','//*[@id="pendo-close-guide-2e51a252"]').click()

    driver.implicitly_wait(15) 

    for journal in journals:
        #clear journal name and change it
        driver.find_element('xpath','//*[@id="pendo-main-search-bar"]/div/div/label/input').clear()
        driver.find_element('xpath','//*[@id="pendo-main-search-bar"]/div/div/label/input').send_keys(f'"{journal}"')

        driver.implicitly_wait(15) 
        #open source title filter
        driver.find_element('xpath','//*[@id="container"]/micro-ui/document-search-results-page/div[1]/section[2]/div/div[1]/div[2]/div/div[2]/div/els-focus-tab-detector[8]/div/button').click()
        driver.find_element('xpath', '//*[@id="container"]/micro-ui/document-search-results-page/div[1]/section[1]/div[1]/div[2]/div/form/div[2]/div[2]/button[2]').click()
        driver.implicitly_wait(15) 
        
        #select journal
        #print(journal)
        #driver.find_element('name', journal).click()
        #driver.find_element('xpath','//*[@id="container"]/micro-ui/document-search-results-page/div[1]/section[2]/div/div[1]/div[2]/div/div[2]/div/div[1]/div/div/button[2]/span').click()


        #get number of publications in filter
        #driver.implicitly_wait(15)
        time.sleep(3)
        n_publications = driver.find_elements(By.TAG_NAME, 'h2')[0].text.replace(',', '')
        n_publications = int(n_publications.replace(' documents found', ''))

        for page in range(math.ceil(n_publications/20_000)):
            try:
                # Attempt to switch to the alert
                alert = driver.switch_to.alert

                # If successful, print the text and accept it
                alert_text = alert.text
                alert.accept()

            except NoAlertPresentException:
                print('no alert')

            last_article = int(str(page+1)+'000')*20

            #click export button, choose csv
            driver.find_element('xpath', '//*[@id="container"]/micro-ui/document-search-results-page/div[1]/section[2]/div/div[2]/div/div[2]/div/div[1]/table/tbody/tr/td[2]/div/div/div[1]/span/button').click()
            driver.implicitly_wait(15) 
            driver.find_elements(By.CLASS_NAME, "Typography-module__lVnit")[52].click()


            #choose range
            driver.implicitly_wait(15)
            driver.find_element('xpath', "//input[@type='radio' and @name='select-current-page' and not(@checked)]").click()
            #start
            driver.find_element('xpath', '//*[@id="container"]/micro-ui/document-search-results-page/div[1]/section[2]/div/div[2]/div/div[2]/div/div[1]/table/tbody/tr/td[2]/div/div/div[2]/div/div/section/div[1]/div/div/div[1]/div/div/div/div/div/div/div[1]/div/label/input').clear()
            driver.find_element('xpath', '//*[@id="container"]/micro-ui/document-search-results-page/div[1]/section[2]/div/div[2]/div/div[2]/div/div[1]/table/tbody/tr/td[2]/div/div/div[2]/div/div/section/div[1]/div/div/div[1]/div/div/div/div/div/div/div[1]/div/label/input').send_keys(last_article-19_999)
            #end
            if page==(math.ceil(n_publications/20_000)-1):
                last_article=n_publications
            driver.find_element('xpath', '//*[@id="container"]/micro-ui/document-search-results-page/div[1]/section[2]/div/div[2]/div/div[2]/div/div[1]/table/tbody/tr/td[2]/div/div/div[2]/div/div/section/div[1]/div/div/div[1]/div/div/div/div/div/div/div[2]/div/label/input').clear()
            driver.find_element('xpath', '//*[@id="container"]/micro-ui/document-search-results-page/div[1]/section[2]/div/div[2]/div/div[2]/div/div[1]/table/tbody/tr/td[2]/div/div/div[2]/div/div/section/div[1]/div/div/div[1]/div/div/div/div/div/div/div[2]/div/label/input').send_keys(last_article)
            
            # Select all information
            driver.find_element('xpath', '//*[@id="container"]/micro-ui/document-search-results-page/div[1]/section[2]/div/div[2]/div/div[2]/div/div[1]/table/tbody/tr/td[2]/div/div/div[2]/div/div/section/div[2]/div/div/span[1]/div/button/span').click()

            # export all information
            driver.implicitly_wait(10)
            driver.find_element('xpath', '//*[@id="container"]/micro-ui/document-search-results-page/div[1]/section[2]/div/div[2]/div/div[2]/div/div[1]/table/tbody/tr/td[2]/div/div/div[2]/div/div/section/div[2]/div/div/span[2]/div/div/button/span[1]/div').click()

            try:
                success_message = WebDriverWait(driver, 1000).until(
                    EC.presence_of_element_located((By.CSS_SELECTOR, "[data-testid='file-type-success-message']"))
                )

                # If the success message is found, continue with the next task
                print(f"Success message found {loop_count}:", success_message.text)
                time.sleep(4)

            except Exception as e:
                # If the success message is not found within the specified time, handle the exception
                print("No success message found:", str(e))

            import the file and concatenate to create a single dataframe
            if loop_count == 0:
                file_path = 'C:/Users/isabe/Downloads/scopus.csv'
                df = pd.read_csv(file_path)
                
            else:
                file_path = f'C:/Users/isabe/Downloads/scopus ({loop_count}).csv'
                df = pd.read_csv(file_path)
                
            loop_count += 1
            concat_df = pd.concat([concat_df, df])

    #driver.close()
    return concat_df

In [20]:
df_0 = import_scopus(link, list_of_journals[0])

no alert
Success message found 0: Your CSV file was successfully exported.
no alert
Success message found 0: Your CSV file was successfully exported.
no alert


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//input[@type='radio' and @name='select-current-page' and not(@checked)]"}
  (Session info: chrome=116.0.5845.188); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x004637C3+48947]
	(No symbol) [0x003F8551]
	(No symbol) [0x002FC92D]
	(No symbol) [0x00329E38]
	(No symbol) [0x00329EFB]
	(No symbol) [0x00358EF2]
	(No symbol) [0x003450D4]
	(No symbol) [0x003575DA]
	(No symbol) [0x00344E86]
	(No symbol) [0x003216C7]
	(No symbol) [0x0032284D]
	GetHandleVerifier [0x006AFDF9+2458985]
	GetHandleVerifier [0x006F744F+2751423]
	GetHandleVerifier [0x006F1361+2726609]
	GetHandleVerifier [0x004E0680+560624]
	(No symbol) [0x0040238C]
	(No symbol) [0x003FE268]
	(No symbol) [0x003FE392]
	(No symbol) [0x003F10B7]
	BaseThreadInitThunk [0x77347BA9+25]
	RtlInitializeException

In [39]:
driver.find_element('xpath', '//*[@id="container"]/micro-ui/document-search-results-page/div[1]/section[2]/div/div[2]/div/div[2]/div/div[1]/table/tbody/tr/td[2]/div/div/div[1]/span/button').click()
driver.find_elements(By.CLASS_NAME, "Typography-module__lVnit")[52].click()
# driver.implicitly_wait(15) 
elem= driver.find_elements(By.CLASS_NAME, "Typography-module__lVnit")[52]
print(elem.text)
# elem.click()

Learn more (opens in a new window)


In [44]:
# Find elements with the specified class name
elements = driver.find_elements(By.CLASS_NAME, "Typography-module__lVnit")

In [45]:
elements[52].text

'CSV'

In [21]:
for i in elements:
    print(i.text)

Universidade Nova de Lisboa
Opens in a new tab
Opens in a new tab

Elsevier Logo

Scopus Logo
Search
Lists
Sources
SciVal
Opens in a new tab
How can we help?
Brought to you by
Create account
Sign in

Welcome to a more intuitive and efficient search experience.
See what is new
Save search
Set search alert
Search within
Clear
Add search field
Reset
Search
Documents
Patents
Secondary documents
Research data
Research data
3,871 documents found
Analyze results
Analyze results
Refine search
To focus on the results area press Control + Shift + Y.
To focus on the results area press Control + Shift + Y.
Filters
. To focus on the results area press Control + Shift + Y.
. To focus on the results area press Control + Shift + Y.
Range
Individual

Show all
Show all



Export filter counts
Search results area. To focus on the filters area press Control + Shift + F.

All
Export
File types
CSV
RIS
BibTeX
Plain text
Reference managers
Mendeley
Zotero (RIS)
EndNote (RIS)
Platforms
SciVal
Download
Citatio

In [13]:
from selenium import webdriver
from selenium.webdriver.common.by import By



# Find the element with the text 'CSV'
desired_text = 'CSV'
found_element = None

for element in elements:
    if desired_text in element.text:
        found_element = element
        break

# Now, 'found_element' should contain the desired element
if found_element:
    print(f"Element with text '{desired_text}' found!")
    # Do something with the found element, e.g., click it
    found_element.click()
else:
    print(f"Element with text '{desired_text}' not found.")


Element with text 'CSV' not found.


In [ ]:
"""

#open browser
service = Service(ChromeDriverManager().install()) #install the chrome driver correspondent to current chrome version
driver = webdriver.Chrome(service= service)
wait = WebDriverWait(driver, 20)

#go to website
driver.get(link) #link should be the link of the query in wos
driver.implicitly_wait(15)

#close pop-up
driver.find_element('xpath','//*[@id="pendo-close-guide-2e51a252"]').click()

driver.implicitly_wait(15) 

#clear journal name and change it
driver.find_element('xpath','//*[@id="pendo-main-search-bar"]/div/div/label/input').clear()
driver.find_element('xpath','//*[@id="pendo-main-search-bar"]/div/div/label/input').send_keys('"American Journal of Hematology"')

driver.implicitly_wait(15) 
#open source title filter
driver.find_element('xpath','//*[@id="container"]/micro-ui/document-search-results-page/div[1]/section[2]/div/div[1]/div[2]/div/div[2]/div/els-focus-tab-detector[8]/div/button').click()

driver.implicitly_wait(15) 
#select journal
driver.find_element('name',"American Journal Of Hematology").click()
driver.find_element('xpath','//*[@id="container"]/micro-ui/document-search-results-page/div[1]/section[2]/div/div[1]/div[2]/div/div[2]/div/div[1]/div/div/button[2]/span').click()


#get number of publications in filter
#driver.implicitly_wait(15)
time.sleep(5)
n_publications = driver.find_elements(By.TAG_NAME, 'h2')[0].text.replace(',', '')
n_publications = int(n_publications.replace(' documents found', ''))

#click export button, choose csv
driver.find_element('xpath', '//*[@id="container"]/micro-ui/document-search-results-page/div[1]/section[2]/div/div[2]/div/div[2]/div/div[1]/table/tbody/tr/td[2]/div/div/div[1]/span/button').click()
driver.implicitly_wait(15) 
driver.find_elements(By.CLASS_NAME, "Typography-module__lVnit")[54].click()


#choose range
driver.implicitly_wait(15)
driver.find_element('xpath', "//input[@type='radio' and @name='select-current-page' and not(@checked)]").click()
#start
driver.find_element('xpath', '//*[@id="container"]/micro-ui/document-search-results-page/div[1]/section[2]/div/div[2]/div/div[2]/div/div[1]/table/tbody/tr/td[2]/div/div/div[2]/div/div/section/div[1]/div/div/div[1]/div/div/div/div/div/div/div[1]/div/label/input').clear()
driver.find_element('xpath', '//*[@id="container"]/micro-ui/document-search-results-page/div[1]/section[2]/div/div[2]/div/div[2]/div/div[1]/table/tbody/tr/td[2]/div/div/div[2]/div/div/section/div[1]/div/div/div[1]/div/div/div/div/div/div/div[1]/div/label/input').send_keys(1)
#end
driver.find_element('xpath', '//*[@id="container"]/micro-ui/document-search-results-page/div[1]/section[2]/div/div[2]/div/div[2]/div/div[1]/table/tbody/tr/td[2]/div/div/div[2]/div/div/section/div[1]/div/div/div[1]/div/div/div/div/div/div/div[2]/div/label/input').clear()
driver.find_element('xpath', '//*[@id="container"]/micro-ui/document-search-results-page/div[1]/section[2]/div/div[2]/div/div[2]/div/div[1]/table/tbody/tr/td[2]/div/div/div[2]/div/div/section/div[1]/div/div/div[1]/div/div/div/div/div/div/div[2]/div/label/input').send_keys(n_publications)
# Select all information
driver.find_element('xpath', '//*[@id="container"]/micro-ui/document-search-results-page/div[1]/section[2]/div/div[2]/div/div[2]/div/div[1]/table/tbody/tr/td[2]/div/div/div[2]/div/div/section/div[2]/div/div/span[1]/div/button/span').click()

# export all information
driver.implicitly_wait(10)
driver.find_element('xpath', '//*[@id="container"]/micro-ui/document-search-results-page/div[1]/section[2]/div/div[2]/div/div[2]/div/div[1]/table/tbody/tr/td[2]/div/div/div[2]/div/div/section/div[2]/div/div/span[2]/div/div/button/span[1]/div').click()

#<span class="Typography-module__lVnit Typography-module__Nfgvc Button-module__Imdmt">Select all information</span>

try:
    success_message = WebDriverWait(driver, 1000).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, "[data-testid='file-type-success-message']"))
    )

    # If the success message is found, continue with the next task
    print("Success message found:", success_message.text)

    # Close pop up
    button = driver.find_element(By.XPATH, "//button[@class='Toastify__close-button Toastify__close-button--dark']")
    driver.execute_script("arguments[0].click();", button)

    # Clear filter
    driver.find_element('xpath', '//*[@id="container"]/micro-ui/document-search-results-page/div[1]/section[2]/div/div[1]/div[2]/div/div[2]/div/els-focus-tab-detector[8]/div/button/button/span').click()
    #<svg aria-hidden="true" viewBox="0 0 14 16" class=""><path fill-rule="evenodd" d="M7.71 8.23l3.75 3.75-1.48 1.48-3.75-3.75-3.75 3.75L1 11.98l3.75-3.75L1 4.48 2.48 3l3.75 3.75L9.98 3l1.48 1.48-3.75 3.75z"></path></svg>
except Exception as e:
    # If the success message is not found within the specified time, handle the exception
    print("No success message found:", str(e))


"""

Success message found: Your CSV file was successfully exported.
